In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, plot_confusion_matrix
from sklearn.metrics import balanced_accuracy_score

In [3]:
import pandas
Data = pandas.read_csv('Cleaned_Data.csv')
Data = Data.drop(['Unnamed: 0'] , axis =1)
Data = Data.dropna()
Data

file content  class number
0      As summer the usual are being made Me was thin...             0
1      Southern 41493 Please send me any to this ride...             0
2      1 Trying to figure out a way to put a halogen ...             0
3      In article Frank Ball wrote The 400 model is e...             0
4      Eric Nelson My 83 Nighthawk two related with t...             0
...                                                  ...           ...
18743  12GB hard Drive Brand NEW with full factory wa...            19
18744  my 14 monitor id dead due to the failure if yo...            19
18745  selling 388 worth of for 100 or Ill split it i...            19
18746  Computer card good for doing graphics on your ...            19
18747  COMPUTER HARDWARE all are working fine last ti...            19

[18707 rows x 2 columns]

In [4]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(Data['file content'], Data['class number'])

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

# Feature engineering


### Counter Vectors

In [5]:
# create a count vectorizer object 
cv = CountVectorizer( token_pattern=r'\b[^\d\W]{2,}\b', stop_words='english')
cv.fit(Data['file content'])

traiv_cv =  cv.transform(train_x)
test_cv =  cv.transform(test_x)

### TF-IDF

In [6]:
# Perform tf-idf on the word level
tfidf_word_level = TfidfVectorizer(token_pattern=r'\b[^\d\W]{2,}\b', stop_words='english', max_features=5000)
tfidf_word_level.fit(Data['file content'])

train_tfidf_word_level =  tfidf_word_level.transform(train_x)
test_tfidf_word_level  =  tfidf_word_level.transform(test_x)

In [7]:
# Perform tf-idf on the n-gram level
tfidf_ngram_level = TfidfVectorizer(token_pattern=r'\b[^\d\W]{2,}\b', stop_words='english', ngram_range=(2,3), max_features=5000)
tfidf_ngram_level.fit(Data['file content'])

train_tfidf_ngram_level =  tfidf_ngram_level.transform(train_x)
test_tfidf_ngram_level =  tfidf_ngram_level.transform(test_x)

In [8]:
# Perform tf-idf on the n-gram level
tfidf_char_ngram_level = TfidfVectorizer(analyzer='char' , stop_words='english', ngram_range=(2,3), max_features=5000)
tfidf_char_ngram_level.fit(Data['file content'])

train_tfidf_char_ngram_level =  tfidf_char_ngram_level.transform(train_x)
test_tfidf_char_ngram_level =  tfidf_char_ngram_level.transform(test_x)

/home/mg/miniconda3/envs/NLP/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:502: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"


# Supervised Learning

We will use the following models :

    1.Naive Bayes Classifier
    2.Linear Classifier
    3.Support Vector Machine

In [10]:
def train_test(classifier, feature_vector_train, label,feature_vector_test , test_label ):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label )
    

    # predict the labels on test dataset
    test_predictions = classifier.predict(feature_vector_test)
    TestAccuracy =metrics.accuracy_score(test_predictions, test_label)
   
    ConfusionMatrix = confusion_matrix(test_label, test_predictions)
    PrecisionScore = precision_score(test_label, test_predictions ,average='macro')
    RecallScore = recall_score(test_label, test_predictions ,average='micro')
    
    return TestAccuracy , PrecisionScore , RecallScore

### Naive Bayes

In [15]:
# NB & CV
metrics_cv = train_test(naive_bayes.MultinomialNB(), traiv_cv, train_y, test_cv , test_y)

# NB & tf-idf on the word level
metrics_tfidf_word_level = train_test(naive_bayes.MultinomialNB(), train_tfidf_word_level, train_y, test_tfidf_word_level , test_y)

# NB & tf-idf on the n-gram level
metrics_tfidf_ngram_level = train_test(naive_bayes.MultinomialNB(), train_tfidf_ngram_level, train_y, test_tfidf_ngram_level , test_y)

# NB & tf-idf on the n-gram level
metrics_tfidf_char_ngram_level = train_test(naive_bayes.MultinomialNB(), train_tfidf_char_ngram_level, train_y, test_tfidf_char_ngram_level , test_y)


/home/mg/miniconda3/envs/NLP/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
print(metrics_cv)
print(metrics_tfidf_word_level)
print(metrics_tfidf_ngram_level)
print(metrics_tfidf_char_ngram_level)

(0.77228992944195, 0.7766772529593127, 0.77228992944195)
(0.7577506948898867, 0.772897143459842, 0.7577506948898867)
(0.5467179816121446, 0.5917871188589537, 0.5467179816121446)
(0.6053025443660466, 0.6460662386906222, 0.6053025443660466)


### Linear Classifier

In [17]:
# LC & CV
metrics_cv = train_test(linear_model.LogisticRegression(), traiv_cv, train_y, test_cv , test_y)

# LC & tf-idf on the word level
metrics_tfidf_word_level = train_test(linear_model.LogisticRegression(), train_tfidf_word_level, train_y, test_tfidf_word_level , test_y)

# LC & tf-idf on the n-gram level
metrics_tfidf_ngram_level = train_test(linear_model.LogisticRegression(), train_tfidf_ngram_level, train_y, test_tfidf_ngram_level , test_y)

# LC & tf-idf on the n-gram level
metrics_tfidf_char_ngram_level = train_test(linear_model.LogisticRegression(), train_tfidf_char_ngram_level, train_y, test_tfidf_char_ngram_level , test_y)


/home/mg/miniconda3/envs/NLP/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mg/miniconda3/envs/NLP/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

In [18]:
print(metrics_cv)
print(metrics_tfidf_word_level)
print(metrics_tfidf_ngram_level)
print(metrics_tfidf_char_ngram_level)

(0.7598888176181313, 0.7595466493138123, 0.7598888176181313)
(0.7757109258071413, 0.7732988742563813, 0.7757109258071413)
(0.5531323497968783, 0.5895820675899827, 0.5531323497968783)
(0.6953175112251443, 0.6927896746815988, 0.6953175112251443)


### SVM

In [ ]:
# SVM & CV
metrics_cv = train_test(svm.SVC(), traiv_cv, train_y, test_cv , test_y)

# SVM & tf-idf on the word level
metrics_tfidf_word_level = train_test(svm.SVC(), train_tfidf_word_level, train_y, test_tfidf_word_level , test_y)

# SVM & tf-idf on the n-gram level
metrics_tfidf_ngram_level = train_test(svm.SVC(), train_tfidf_ngram_level, train_y, test_tfidf_ngram_level , test_y)

# SVM & tf-idf on the n-gram level
metrics_tfidf_char_ngram_level = train_test(svm.SVC(), train_tfidf_char_ngram_level, train_y, test_tfidf_char_ngram_level , test_y)


# Unsupervised Learning

### KNeighborsClassifier

In [19]:
# KNeighborsClassifier & CV
metrics_cv = train_test(KNeighborsClassifier(), traiv_cv, train_y, test_cv , test_y)

# KNeighborsClassifier & tf-idf on the word level
metrics_tfidf_word_level = train_test(KNeighborsClassifier(), train_tfidf_word_level, train_y, test_tfidf_word_level , test_y)

# KNeighborsClassifier & tf-idf on the n-gram level
metrics_tfidf_ngram_level = train_test(KNeighborsClassifier(), train_tfidf_ngram_level, train_y, test_tfidf_ngram_level , test_y)

# KNeighborsClassifier & tf-idf on the n-gram level
metrics_tfidf_char_ngram_level = train_test(KNeighborsClassifier(), train_tfidf_char_ngram_level, train_y, test_tfidf_char_ngram_level , test_y)


In [20]:
print(metrics_cv)
print(metrics_tfidf_word_level)
print(metrics_tfidf_ngram_level)
print(metrics_tfidf_char_ngram_level)

(0.4111610006414368, 0.5389636427364759, 0.4111610006414368)
(0.3068206115031003, 0.7442283979643646, 0.3068206115031003)
(0.3639084883472311, 0.41836890254760706, 0.3639084883472311)
(0.5116527688689331, 0.6151275917099144, 0.5116527688689331)


# Ensembling

### Random Forest

In [13]:
# RF & CV
metrics_cv = train_test(ensemble.RandomForestClassifier(), traiv_cv, train_y, test_cv , test_y)

# RF & tf-idf on the word level
metrics_tfidf_word_level = train_test(ensemble.RandomForestClassifier(), train_tfidf_word_level, train_y, test_tfidf_word_level , test_y)


In [14]:
print(metrics_cv)
print(metrics_tfidf_word_level)

(0.7237545435107975, 0.7344042704116588, 0.7237545435107975)
(0.6968141971349155, 0.6995387038626728, 0.6968141971349155)


### XGBoosting

In [11]:
# XGBoost & CV
metrics_cv = train_test(xgboost.XGBClassifier(), traiv_cv.tocsc(), train_y, test_cv.tocsc() , test_y)

# XGBoost & tf-idf on the word level
metrics_tfidf_word_level = train_test(xgboost.XGBClassifier(), train_tfidf_word_level.tocsc(), train_y, test_tfidf_word_level.tocsc() , test_y)


/home/mg/miniconda3/envs/NLP/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:00:08] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/mg/miniconda3/envs/NLP/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:00:47] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [12]:
print(metrics_cv)
print(metrics_tfidf_word_level)

(0.7258926662390421, 0.7245284226994679, 0.7258926662390421)
(0.7055805003207184, 0.7014430990396145, 0.7055805003207184)
